<a href="https://colab.research.google.com/github/cardstdani/practica-par/blob/main/PracticaParadigmas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Práctica Paradigmas 1**

In [ ]:
!pip install pyvis==0.3.1

In [43]:
import random
import re
import numpy as np
import pandas as pd
from IPython.display import display, HTML
from pyvis.network import Network

class TrieNode:
    def __init__(self, inputChar):
        self.char = inputChar
        self.end = False
        self.children = {}
 
class Trie():
    def __init__(self, startingElements=None):
        self.root = TrieNode("")
        if startingElements!=None:
          for i in startingElements: self.insert(i)
    def insert(self, word):
        node = self.root
        for char in word:
            if char in node.children:
                node = node.children[char]
            else:
                new_node = TrieNode(char)
                node.children[char] = new_node
                node = new_node
        node.end = True      
    def dfs(self, node, pre):
        if node.end:
            self.output.append((pre + node.char))
        for child in node.children.values():
            self.dfs(child, pre + node.char)
    def search(self, x):
        node = self.root
        for char in x:
            if char in node.children:
                node = node.children[char]
            else:
                return []
        self.output = []
        self.dfs(node, x[:-1])
        return self.output
    def searchAndSplit(self, x):
        node = self.root
        output = ["", ""]
        for char in x:
            if char in node.children:
                node = node.children[char]            
            else:
                return []        
            output[1 if node.end else 0] += node.char
        return output if node.end else []
    def toGraph(self):
      g = Network(directed =True)      
      g.show_buttons()

      nodeIndex = 1
      currentNode = 0
      q = [self.root]      
      g.add_node(currentNode, label="", color="red")
      tempLabels = {0:""}
      while q!=[]:
        n = q.pop(0)                     
        for i in n.children.values():
          tempLabels[nodeIndex] = tempLabels[currentNode]+i.char
          g.add_node(nodeIndex, label=tempLabels[currentNode]+i.char, color="#48e073" if i.end else "blue")
          g.add_edge(currentNode, nodeIndex)
          nodeIndex+=1
          q.append(i)
        currentNode+=1
      g.show('nx.html')

class MainGame:
  def __init__(self):
    self.matrix = [random.sample(["."]*45+["a"]*18+["b"]*4+["c"]*3+["1"]*2, 6) for i in range(6)]
    self.turn = 0
    self.score = 0
    self.storage = "."
    self.actual = random.choice(["a"]*30+["b"]*5+["c"]*1+["1"]*6+["w"]*1)
    self.tr = Trie((''.join(chr(97+int(j)) for j in str(i))+str(k) for k in range(len(self.matrix[0])) for i in range(len(self.matrix))))
    self.tr.insert("exit")
    self.tr.insert("*")

  def main(self):
    self.showGame()
    self.tr.toGraph()
    while any("." in x for x in self.matrix):
      message = self.validarEntrada(input("Mover a casilla: ").lower().replace(" ", ""))
      while not message[0]: message = self.validarEntrada(input("Jugada errónea\nMover a casilla: ").lower().replace(" ", ""))
      if "".join(message[1])=="exit": break
      if "".join(message[1])=="*": self.storage=self.actual; self.showGame(); continue
      coordinates = (lambda l: [int("".join([str(ord(i)%97) for i in l[0]])), int(l[1])])(message[1])
      #check
      print(coordinates) 

      self.updateActual()
      self.turn+=1
      self.showGame()
  def updateActual(self):
    self.actual = random.choice(["a"]*30+["b"]*5+["c"]*1+["1"]*6+["w"]*1)   
  
  def validarEntrada(self, s):
    s = self.tr.searchAndSplit(s)
    return (True if s else False, s)

  def showGame(self):
    df = pd.DataFrame(self.matrix, columns=list(range(len(self.matrix[0]))), index=[''.join(chr(65+int(j)) for j in str(i)) for i in range(len(self.matrix))])
    display(HTML(df.to_html()))
    print(f"\nTurno: {self.turn} Puntos:{self.score}\nAlmacen: [{self.storage}] Actual: [{self.actual}]")

if __name__=="__main__":
  g = MainGame()
  g.main()

,0,1,2,3,4,5
A,a,.,.,a,.,a
B,.,.,.,a,.,a
C,1,.,b,b,a,.
D,a,.,.,a,.,.
E,a,b,.,a,a,a
F,.,a,.,.,.,.



Turno: 0 Puntos:0
Almacen: [.] Actual: [a]
Mover a casilla: a4
[0, 4]


,0,1,2,3,4,5
A,a,.,.,a,.,a
B,.,.,.,a,.,a
C,1,.,b,b,a,.
D,a,.,.,a,.,.
E,a,b,.,a,a,a
F,.,a,.,.,.,.



Turno: 1 Puntos:0
Almacen: [.] Actual: [a]
Mover a casilla: a9
Jugada errónea
Mover a casilla: 9
Jugada errónea
Mover a casilla: 4a
Jugada errónea
Mover a casilla: d2
[3, 2]


,0,1,2,3,4,5
A,a,.,.,a,.,a
B,.,.,.,a,.,a
C,1,.,b,b,a,.
D,a,.,.,a,.,.
E,a,b,.,a,a,a
F,.,a,.,.,.,.



Turno: 2 Puntos:0
Almacen: [.] Actual: [1]
Mover a casilla: exit
